<a href="https://colab.research.google.com/github/nyctzo/IMAGE-CLASSIFICATION/blob/main/IMAGE_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle= True, num_workers=2)

In [ ]:
image, label = train_data[0]


In [ ]:
image


tensor([[[-0.5373, -0.6627, -0.6078,  ...,  0.2392,  0.1922,  0.1608],
         [-0.8745, -1.0000, -0.8588,  ..., -0.0353, -0.0667, -0.0431],
         [-0.8039, -0.8745, -0.6157,  ..., -0.0745, -0.0588, -0.1451],
         ...,
         [ 0.6314,  0.5765,  0.5529,  ...,  0.2549, -0.5608, -0.5843],
         [ 0.4118,  0.3569,  0.4588,  ...,  0.4431, -0.2392, -0.3490],
         [ 0.3882,  0.3176,  0.4039,  ...,  0.6941,  0.1843, -0.0353]],

        [[-0.5137, -0.6392, -0.6235,  ...,  0.0353, -0.0196, -0.0275],
         [-0.8431, -1.0000, -0.9373,  ..., -0.3098, -0.3490, -0.3176],
         [-0.8118, -0.9451, -0.7882,  ..., -0.3412, -0.3412, -0.4275],
         ...,
         [ 0.3333,  0.2000,  0.2627,  ...,  0.0431, -0.7569, -0.7333],
         [ 0.0902, -0.0353,  0.1294,  ...,  0.1608, -0.5137, -0.5843],
         [ 0.1294,  0.0118,  0.1137,  ...,  0.4431, -0.0745, -0.2784]],

        [[-0.5059, -0.6471, -0.6627,  ..., -0.1529, -0.2000, -0.1922],
         [-0.8431, -1.0000, -1.0000,  ..., -0

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class NeuralNet(nn.Module):


 def __init__ (self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 12, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(12, 24, 5)
    self.fc1  = nn.Linear(24 * 5 * 5, 120)
    self.fc2  = nn.Linear(120, 84)
    self.fc3  = nn.Linear(84, 10)

 def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
net = NeuralNet()
LOSS = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr= 0.001, momentum = 0.9)



In [ ]:
for epoch in range(30):
  print(f"training epoch: {epoch}....")

  running_loss = 0.0
  for i, data in enumerate(train_loader):
      inputs, labels = data

      optimizer.zero_grad()
      outputs = net(inputs)

      loss = LOSS(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()

  print(f"loss: {running_loss / len(train_loader):.4f}")


training epoch: 0....
loss: 1.1563
training epoch: 1....
loss: 1.0221
training epoch: 2....
loss: 0.9227
training epoch: 3....
loss: 0.8504
training epoch: 4....
loss: 0.7944
training epoch: 5....
loss: 0.7423
training epoch: 6....
loss: 0.7042
training epoch: 7....
loss: 0.6622
training epoch: 8....
loss: 0.6295
training epoch: 9....
loss: 0.5981
training epoch: 10....
loss: 0.5722
training epoch: 11....
loss: 0.5589
training epoch: 12....
loss: 0.5290
training epoch: 13....
loss: 0.5079
training epoch: 14....
loss: 0.5012
training epoch: 15....
loss: 0.4820
training epoch: 16....
loss: 0.4748
training epoch: 17....
loss: 0.4607
training epoch: 18....
loss: 0.4532
training epoch: 19....
loss: 0.4425
training epoch: 20....
loss: 0.4322
training epoch: 21....
loss: 0.4288
training epoch: 22....
loss: 0.4309
training epoch: 23....
loss: 0.4257
training epoch: 24....
loss: 0.4243
training epoch: 25....
loss: 0.4109
training epoch: 26....
loss: 0.4196
training epoch: 27....
loss: 0.4195
tr

In [ ]:
torch.save(net.state_dict(), "trained_net.path")

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load("trained_net.path"))

<All keys matched successfully>

In [ ]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
  for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f"Accuracy: {accuracy}%")

Accuracy: 65.33%


In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5) , (0.5, 0.5, 0.5))]
)

def load_image (image_path):
  image = Image.open(image_path)
  image = new_transform(image)
  image = image.unsqueeze(0)
  return image

image_path = ["/content/PLANE.JPG",]
images = [load_image(img) for img in image_path]

net.eval()
with torch.no_grad():
  for image in images:
    output = net(image)
    _, predicted = torch.max(output.data, 1)
    print(f"prediction: {class_names[predicted.item()]}")

NameError: name 'transforms' is not defined